In [1]:
!pip install pytesseract pillow opencv-python -q

In [2]:
import pytesseract
import PIL
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# files = []
# for i in range(1, 16):
#   if i < 10:
#     path = f"Search Report_page-{'000' + str(i)}.jpg"
#   else:
#     path = f"Search Report_page-{'00' + str(i)}.jpg"
#   files.append(path)

# files_it = iter(files)
# len(files)

In [4]:
# fig, ax = plt.subplots(3, 5, figsize = (20 , 20))
# # from google.colab.patches import cv2_imshow
# count = 0
# for i in range(3):
#   for j in range(5):
#     if count < 15:
#       img = files[count]
#       img = plt.imread(img)
#       ax[i][j].imshow(img)
#       count = count + 1


# **Page - 3 Text preprocessing**

In [5]:
text = """
SCHEDULE-I
(Details of Developer and Landowners)

Name & Address of the Developer :

M/s Harshit Singhania Buildcon through its partner
Singhania Buildcon Pvt. Ltd. through its Director Shri
Subodh Singhania S/o Shri Shyam Lal Singhania,
Registered Office Amanaka, G.E. Road, Raipur Tahsil
& District Raipur (C.G.)

Name & Address of the Landowner
(Present Owner of the immovable
property) :

M/s Harshit Singhania Buildcon through its partner
Singhania Buildcon Pvt. Ltd. through its Director Shri
Subodh Singhania S/o Shri Shyam Lal Singhania,
Registered Office Amanaka, G.E. Road, Raipur Tahsil
& District Raipur (C.G.)

Address of the Property under the
project:

Situated at Known as �Harshit Neo City� Village
Amleshwar, P.C. No. 05, R.I.C. Bhilai-3, Tahsil Patan,
District Durg (C.G.)

(Details of I

SCHEDULE-II

mmovable Property)

Plot No. / Land Revenue Survey Nos.
Kh. No with its Area covered under
Title scrutiny:

Diverted Land bearing of Khasra Nos. 208, 212, 213,
215, 216 Comprising Area 0.84, 2.01, 2.96, 1.00, 1.00
Hect., respectively, Total Area 7.81 Hect., or 8,40,356.
Sq. Ft., (Sale out 227 Plots)

Detailed Address of aforesaid
Land/Plot: (village, Patwari Halka no.,
Block, Municipal Ward No., Tehsil,
District, State, Pincode)

Situated at Known as �Harshit Neo City� Village
Amleshwar, P.C. No. 05, R.I.C. Bhilai-3, Tahsil Patan,
District Durg (C.G.)

Boundaries

__As Per Sale Deed
NORTH Land of Harshit Singhania Buildcon
SOUTH Land bearing Kh. No. 230, 231
EAST Land of Harshit Singhania Buildcon

WEST Land bearing Kh. No. 217, 218

Nature/Type of Land/Plot

Residential/Cum Commercial

* Description of Documents Scrutinized & Ve

tified:

Ihave examined & verified the documents as mentioned in the Schedule-III attached herewith.
"""

In [6]:
text_split = text.split("\n\n")

In [7]:
text_split

['\nSCHEDULE-I\n(Details of Developer and Landowners)',
 'Name & Address of the Developer :',
 'M/s Harshit Singhania Buildcon through its partner\nSinghania Buildcon Pvt. Ltd. through its Director Shri\nSubodh Singhania S/o Shri Shyam Lal Singhania,\nRegistered Office Amanaka, G.E. Road, Raipur Tahsil\n& District Raipur (C.G.)',
 'Name & Address of the Landowner\n(Present Owner of the immovable\nproperty) :',
 'M/s Harshit Singhania Buildcon through its partner\nSinghania Buildcon Pvt. Ltd. through its Director Shri\nSubodh Singhania S/o Shri Shyam Lal Singhania,\nRegistered Office Amanaka, G.E. Road, Raipur Tahsil\n& District Raipur (C.G.)',
 'Address of the Property under the\nproject:',
 'Situated at Known as �Harshit Neo City� Village\nAmleshwar, P.C. No. 05, R.I.C. Bhilai-3, Tahsil Patan,\nDistrict Durg (C.G.)',
 '(Details of I',
 'SCHEDULE-II',
 'mmovable Property)',
 'Plot No. / Land Revenue Survey Nos.\nKh. No with its Area covered under\nTitle scrutiny:',
 'Diverted Land bear

In [8]:
preprocessed_text_split = []
for sent in text_split:
  x = sent.replace("\n", " ")

  preprocessed_text_split.append(x)

preprocessed_text_split

[' SCHEDULE-I (Details of Developer and Landowners)',
 'Name & Address of the Developer :',
 'M/s Harshit Singhania Buildcon through its partner Singhania Buildcon Pvt. Ltd. through its Director Shri Subodh Singhania S/o Shri Shyam Lal Singhania, Registered Office Amanaka, G.E. Road, Raipur Tahsil & District Raipur (C.G.)',
 'Name & Address of the Landowner (Present Owner of the immovable property) :',
 'M/s Harshit Singhania Buildcon through its partner Singhania Buildcon Pvt. Ltd. through its Director Shri Subodh Singhania S/o Shri Shyam Lal Singhania, Registered Office Amanaka, G.E. Road, Raipur Tahsil & District Raipur (C.G.)',
 'Address of the Property under the project:',
 'Situated at Known as �Harshit Neo City� Village Amleshwar, P.C. No. 05, R.I.C. Bhilai-3, Tahsil Patan, District Durg (C.G.)',
 '(Details of I',
 'SCHEDULE-II',
 'mmovable Property)',
 'Plot No. / Land Revenue Survey Nos. Kh. No with its Area covered under Title scrutiny:',
 'Diverted Land bearing of Khasra Nos

In [12]:
data = {
    "sentences":preprocessed_text_split,
    "title" : np.zeros((len(preprocessed_text_split))),
    "description" : np.zeros((len(preprocessed_text_split))),
    "title + description": np.zeros((len(preprocessed_text_split))),
    "heading" : np.zeros((len(preprocessed_text_split)))
}
data

{'sentences': [' SCHEDULE-I (Details of Developer and Landowners)',
  'Name & Address of the Developer :',
  'M/s Harshit Singhania Buildcon through its partner Singhania Buildcon Pvt. Ltd. through its Director Shri Subodh Singhania S/o Shri Shyam Lal Singhania, Registered Office Amanaka, G.E. Road, Raipur Tahsil & District Raipur (C.G.)',
  'Name & Address of the Landowner (Present Owner of the immovable property) :',
  'M/s Harshit Singhania Buildcon through its partner Singhania Buildcon Pvt. Ltd. through its Director Shri Subodh Singhania S/o Shri Shyam Lal Singhania, Registered Office Amanaka, G.E. Road, Raipur Tahsil & District Raipur (C.G.)',
  'Address of the Property under the project:',
  'Situated at Known as �Harshit Neo City� Village Amleshwar, P.C. No. 05, R.I.C. Bhilai-3, Tahsil Patan, District Durg (C.G.)',
  '(Details of I',
  'SCHEDULE-II',
  'mmovable Property)',
  'Plot No. / Land Revenue Survey Nos. Kh. No with its Area covered under Title scrutiny:',
  'Diverted L

In [13]:
pd.DataFrame(data)

,sentences,title,description,title + description,heading
0,SCHEDULE-I (Details of Developer and Landowners),0.0,0.0,0.0,0.0
1,Name & Address of the Developer :,0.0,0.0,0.0,0.0
2,M/s Harshit Singhania Buildcon through its par...,0.0,0.0,0.0,0.0
3,Name & Address of the Landowner (Present Owner...,0.0,0.0,0.0,0.0
4,M/s Harshit Singhania Buildcon through its par...,0.0,0.0,0.0,0.0
5,Address of the Property under the project:,0.0,0.0,0.0,0.0
6,Situated at Known as �Harshit Neo City� Villag...,0.0,0.0,0.0,0.0
7,(Details of I,0.0,0.0,0.0,0.0
8,SCHEDULE-II,0.0,0.0,0.0,0.0
9,mmovable Property),0.0,0.0,0.0,0.0


In [15]:
!pip install transformers langchain-community -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


In [16]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline
tokenizer = AutoTokenizer.from_pretrained('dslim/bert-base-NER')
model = pipeline('ner', 'dslim/bert-base-NER')
model

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
for sent in preprocessed_text_split:
  print(model(sent))

[{'entity': 'I-ORG', 'score': 0.4940555, 'index': 18, 'word': '##ers', 'start': 45, 'end': 48}]
[]
[{'entity': 'B-ORG', 'score': 0.6262239, 'index': 4, 'word': 'Ha', 'start': 4, 'end': 6}, {'entity': 'I-ORG', 'score': 0.9311767, 'index': 5, 'word': '##rs', 'start': 6, 'end': 8}, {'entity': 'I-ORG', 'score': 0.9805571, 'index': 6, 'word': '##hit', 'start': 8, 'end': 11}, {'entity': 'I-ORG', 'score': 0.9552785, 'index': 7, 'word': 'Singh', 'start': 12, 'end': 17}, {'entity': 'I-ORG', 'score': 0.9779619, 'index': 8, 'word': '##ania', 'start': 17, 'end': 21}, {'entity': 'I-ORG', 'score': 0.9947496, 'index': 9, 'word': 'B', 'start': 22, 'end': 23}, {'entity': 'I-ORG', 'score': 0.9928591, 'index': 10, 'word': '##uild', 'start': 23, 'end': 27}, {'entity': 'I-ORG', 'score': 0.993038, 'index': 11, 'word': '##con', 'start': 27, 'end': 30}, {'entity': 'B-ORG', 'score': 0.9988061, 'index': 15, 'word': 'Singh', 'start': 51, 'end': 56}, {'entity': 'I-ORG', 'score': 0.9907725, 'index': 16, 'word': '#